# Mortgage Amortization Calculator

Welcome to the Mortgage Amortization Calculator! This tool is designed to help you understand and compare the amortization schedules of two distinct mortgage plans. It includes the functionality to consider the effects of lump sum payments and additional monthly payments on the overall interest and the term of the mortgage.

In the following sections, you'll be prompted to enter the details of each mortgage, including the principal amount, the term of the loan, and the interest rate. You'll also have the option to specify lump sum payments and additional monthly payments.

Let's get started by importing the necessary libraries and setting up the initial parameters for our calculations.

## Initial Setup

Let's start by importing all necessary libraries and setting up the initial parameters for our mortgage calculations. Make sure you have the following libraries installed: `numpy`, `matplotlib`, `pandas`, `plotly`, `numpy_financial`, `ipywidgets`, etc.

In [1]:
# Run this cell to install required libraries if they are not already installed
#!pip3 install numpy matplotlib pandas plotly ipywidgets numpy_financial

In [2]:
import datetime
import numpy as np
import numpy_financial as npf
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

## Mortgage Details Input
In this section, you will enter the details for two mortgages. Please provide the original balance, term (in years), and annual interest rate. This data will be used to calculate monthly payments and to generate the amortization schedules.

### Input data for Mortgage 1
Input the details for the first mortgage below.

In [3]:
principal_mortgage1 = float(input("Original balance of Mortgage 1: "))
term_years_mortgage1 = int(input("Term in years of Mortgage 1: "))
annual_rate_percent_mortgage1 = float(input("Annual interest rate (%) of Mortgage 1: "))

Original balance of Mortgage 1:  124104
Term in years of Mortgage 1:  13
Annual interest rate (%) of Mortgage 1:  2.95


### Input data for Mortgage 2
Now, input the details for the second mortgage.

In [4]:
principal_mortgage2 = float(input("Original balance of Mortgage 2: "))
term_years_mortgage2 = int(input("Term in years of Mortgage 2: "))
annual_rate_percent_mortgage2 = float(input("Annual interest rate (%) of Mortgage 2: "))

Original balance of Mortgage 2:  380000
Term in years of Mortgage 2:  27
Annual interest rate (%) of Mortgage 2:  3.75


## Lump Sum Payment

To see the impact of making extra payments towards your mortgage, enter any lump sum payment you plan to make and any additional amount you wish to add to your monthly payments.

In [5]:
# Lump sum payment input
lump_sum = float(input("Amount of Lump Sum Payment for the mortgage (enter 0 if none): "))

Amount of Lump Sum Payment for the mortgage (enter 0 if none):  50000


## Additional Monthly Payment

In case you intend to make extra monthly payments in addition to your regular payments, specify the amount here. These payments contribute to faster mortgage amortization.

In [6]:
# Additional monthly payment input
extra_monthly_payment = float(input("Enter the amount of the extra monthly payment (enter 0 if none): "))

Enter the amount of the extra monthly payment (enter 0 if none):  0


## Mortgage Calculation Functions

This section is dedicated to the functions that are crucial for managing and understanding mortgage finances. Here, we have developed several functions to assist homeowners and potential buyers by calculating monthly mortgage payments, generating detailed amortization schedules, and evaluating the financial benefits of making extra payments. These tools are designed to accommodate various scenarios, including adjustments for extra monthly payments or lump sum contributions, thus offering a clear picture of the financial implications over the life of the mortgage.

### Key Functions:

- **`calc_monthly_payment`:** This function calculates the monthly payment required for a mortgage based on the loan's principal amount, the term of the loan, and the annual interest rate. Understanding this figure is essential for budgeting and financial planning.

- **`amortization_schedule`:** This function generates a comprehensive amortization schedule that outlines how each payment affects the loan balance over time. It includes options to account for additional payments, which can significantly reduce the loan's interest cost and shorten its term.

- **`calculate_remaining_years`:** This function determines how many years are left on a mortgage after accounting for extra payments. It's useful for understanding the impact of making additional payments above the regular monthly amount, including lump sum payments.

- **`calculate_total_interest_saved`:** This function calculates the total interest saved over the life of the mortgage when making extra monthly payments and/or lump sum payments. It helps quantify the financial benefits of paying more than the minimum required each month or making significant one-time payments towards the principal.
- **`calculate_remaining_mortgage`**: Calculates the remaining mortgage balance after accounting for a lump sum payment at the start of the term. Ensures the balance never goes negative, providing a realistic loan balance scenario.

These functions provide essential insights into the dynamics of mortgage amortization, helping you make informed decisions about your home financing or investment strategy. They are invaluable for anyone looking to minimize their interest payments and reduce the term of their mortgage.

In [7]:
def calc_monthly_payment(principal, term_years, annual_rate_percent):
    """
    Calculate the monthly mortgage payment given the principal, term, and annual interest rate.

    Parameters:
    principal (float): The initial loan balance.
    term_years (int): The duration of the loan in years.
    annual_rate_percent (float): The annual interest rate in percentage.

    Returns:
    float: The monthly payment required to amortize the loan over the specified term at the given interest rate.
    """
    if principal <= 0 or term_years <= 0 or annual_rate_percent <= 0:
        raise ValueError("All parameters must be greater than zero.")
    # Calculate the monthly interest rate from the annual percentage rate
    rate_month = annual_rate_percent / 100 / 12
    # Convert the loan term from years to months
    n_months = term_years * 12
    # Compute the compounding term factor
    term = (1 + rate_month) ** n_months
    # Calculate the monthly payment using the formula for an annuity
    return principal * rate_month * term / (term - 1)

In [8]:
def amortization_schedule(principal, term_years, annual_rate_percent, monthly_payment, extra_monthly_payment=0, lump_sum_offset=0, offset_start_year=0, offset_end_year=None):
    """
    Generate an amortization schedule for a mortgage, considering additional payments.

    Parameters:
    principal (float): The initial loan balance.
    term_years (int): The duration of the loan in years.
    annual_rate_percent (float): The annual interest rate in percentage.
    monthly_payment (float): The calculated monthly payment for the mortgage.
    extra_monthly_payment (float): Additional amount paid each month.
    lump_sum_offset (float): Lump sum payment applied directly to the principal.
    offset_start_year (int): The initial year after which the lump sum is applied.
    offset_end_year (int): The final year after which the lump sum is no longer applied.

    Returns:
    tuple: Two numpy arrays where the first is the regular balance and the second considers the additional payment.
    """
    rate_year = annual_rate_percent / 100.
    rate_month = rate_year / 12
    n_months = term_years * 12
    balance = principal
    balance_offset = principal - lump_sum_offset

    schedule = []
    schedule_offset = []

    # Iterate over each month of the loan period
    for month in range(1, n_months + 1):
        # Calculate interest for this month
        interest = balance * rate_month
        interest_offset = balance_offset * rate_month
        # Calculate principal payment
        principal_payment = monthly_payment - interest
        principal_offset = monthly_payment - interest_offset + extra_monthly_payment
        # Update balances
        balance -= principal_payment
        balance_offset -= principal_offset
        
        # Ensure balance does not go negative
        if balance < 0:
            balance = 0
        if balance_offset < 0:
            balance_offset = 0
        
        # Apply extra payments if no offset condition applies
        if lump_sum_offset == 0 or (offset_end_year and month / 12 > offset_end_year):
            balance -= extra_monthly_payment
            if balance < 0:
                balance = 0
        
        # Store the current balances
        schedule.append(balance)
        schedule_offset.append(balance_offset)
        
    return np.array(schedule), np.array(schedule_offset)

In [9]:
def calculate_remaining_years(schedule_with_offset, n_months):
    """
    Calculate the remaining years on the mortgage after applying extra payments.

    Parameters:
    schedule_with_offset (np.array): Amortization schedule considering extra payments.
        This array shows the mortgage balance at each payment interval after applying any additional payments.
    n_months (int): The total number of months originally planned for the mortgage.

    Returns:
    float: The number of years remaining until the mortgage is paid off.
    """
    # Find all months where the mortgage balance is greater than zero
    non_zero_indices = np.nonzero(schedule_with_offset > 0)[0]
    if non_zero_indices.size > 0:
        # The last month with a balance above zero indicates how long the mortgage would last with extra payments
        last_month_with_offset = non_zero_indices[-1] + 1
        # Calculate the number of months remaining by subtracting the last active month from total months
        remaining_months = n_months - last_month_with_offset
        # Convert remaining months to years
        remaining_years = remaining_months / 12
        print(f"Remaining years calculated: {remaining_years} years (Last month with balance: {last_month_with_offset})")
        return remaining_years
    else:
        # If the mortgage is already paid off (no months with balance > 0), return 0 years remaining
        return 0

In [10]:
def calculate_total_interest_saved(principal, interest_rate, years, monthly_payment, extra_monthly_payment=0, lump_sum=0):
    """
    Calculate the total interest saved by making extra monthly payments and/or lump sum payments.

    Parameters:
    principal (float): The original loan amount.
    interest_rate (float): The annual interest rate as a percentage.
    years (int): The original term of the loan in years.
    monthly_payment (float): The regular monthly payment amount.
    extra_monthly_payment (float): Additional monthly amount paid.
    lump_sum (float): Lump sum payment made towards the principal.

    Returns:
    float: Total interest saved by making additional payments.
    """
    # Calculate the total amount of payments and the original interest without any extra payments
    total_payments_original = monthly_payment * years * 12
    total_interest_original = total_payments_original - principal

    # Adjust the principal with any lump sum payment and start recalculating the payments
    remaining_principal = principal - lump_sum
    new_total_payments = 0
    months = 0

    # Continue making payments until the remaining balance is zero
    while remaining_principal > 0:
        # Calculate the interest for the current balance
        interest_payment = remaining_principal * (interest_rate / 100 / 12)
        # Calculate how much of the payment is going towards the principal
        principal_payment = monthly_payment - interest_payment + extra_monthly_payment
        remaining_principal -= principal_payment
        new_total_payments += monthly_payment + extra_monthly_payment
        months += 1
        # Check if the last payment is more than the remaining balance
        if remaining_principal < 0:
            # Reduce the overpayment from the total payments
            overpayment = -remaining_principal
            new_total_payments -= overpayment

    # Calculate the new total interest after extra payments
    total_interest_new = new_total_payments - (principal - lump_sum)
    # Calculate how much interest has been saved with the extra payments
    total_interest_saved = total_interest_original - total_interest_new
    new_years = months / 12
    return total_interest_saved, new_years

In [11]:
def calculate_remaining_mortgage(principal, lump_sum):
    """
    Calculate the remaining mortgage balance after accounting for the lump sum payment.

    Parameters:
    principal (float): The original loan amount.
    lump_sum (float): Lump sum payment made at the start.

    Returns:
    float: The remaining balance of the mortgage after subtracting the lump sum.
    """
    return principal - lump_sum if lump_sum <= principal else 0

## Visualizations and Results

This section leverages the powerful visualization capabilities of Plotly to graphically represent amortization schedules. These visualizations help illustrate how mortgage balances decrease over time under different payment scenarios, including standard monthly payments and scenarios with additional payments. This visual feedback is vital for understanding the impact of overpayments on the principal and the total interest saved over the life of the loan.

### Visualization Features:
- **Interactive Graphs:** Users can interact with the visualization to better understand how each payment affects the loan balance over time.
- **Comparison Views:** It's possible to compare different payment strategies side-by-side to see their long-term financial impact.

In [12]:
def plot_amortization_interactive(schedules, labels, titles):
    """
    Plot interactive amortization schedules for mortgages using Plotly.

    This function creates a two-column subplot for each mortgage, displaying how the mortgage balance decreases over time. 
    It supports visualization of both the regular payment schedule and scenarios with additional payments.

    Parameters:
    schedules (list of tuples): Each tuple contains two numpy arrays; the first array is the regular amortization schedule, 
                                and the second array accounts for additional payments.
    labels (list of tuples): Each tuple contains labels for the plots, typically ('Current Mortgage', 'With Overpayment').
    titles (list): Titles for each subplot, usually indicating different mortgages.

    Returns:
    None: The function directly displays the plot in the Jupyter notebook environment using Plotly.
    """
    # Set the default renderer for Plotly to work in Jupyter notebooks
    pio.renderers.default = 'iframe'
    
    # Create subplot structure with 1 row and as many columns as there are mortgage comparisons
    fig = make_subplots(rows=1, cols=2, subplot_titles=titles, shared_yaxes=False)

    # Loop through each schedule set and plot both the current and overpayment schedules
    for i, (schedule, label, title) in enumerate(zip(schedules, labels, titles), start=1):
        months = len(schedule[0])  # Number of months in the amortization schedule
        years = np.arange(months) / 12  # Convert months to years for the x-axis

        # Plot each type of mortgage balance over time
        for j, balance in enumerate(schedule):
            trace = go.Scatter(
                x=years,
                y=balance / 1000,  # Convert balance to thousands for easier readability
                fill='tozeroy',
                mode='lines',
                name=label[j],
                hoverinfo='y+x',
                hovertemplate=f'<b>{label[j]} Balance:</b> €%{{y:.2f}}k<extra></extra><br><b>Time:</b> %{{x}} years'
            )
            fig.add_trace(trace, row=1, col=i)

        # Update y-axes and x-axes labels for each subplot
        fig.update_yaxes(title_text="Remaining Balance (€k)", row=1, col=i)
        fig.update_xaxes(title_text="Time (years)", row=1, col=i)

    # Update the overall layout settings for better presentation
    fig.update_layout(
        height=500,
        width=1100,
        title_text="Mortgage Amortization Comparison",
        legend=dict(
            x=0.5,
            y=-0.3,
            orientation="h",
            font=dict(size=10),
        ),
        margin=dict(l=50, r=50, b=100, t=100, pad=4)
    )
    # Display the figure in the notebook
    fig.show()

## Data Processing and Visualization

This section outlines the final stages of our mortgage amortization analysis. Here, we focus on refining the data processing steps and generating dynamic visualizations to compare mortgage scenarios effectively.

### Overview:

- **Data Preparation:** Organize all necessary inputs to generate structured outputs, ensuring clarity in the presentation of each mortgage scenario.
- **Summary Tables:** Leverage pandas for constructing readable tables to display summary data, providing quick insights into calculated results.
- **Visualizations:** Employ Plotly for interactive graphs that illustrate amortization schedules, demonstrating the financial impact of different payment strategies.
- **Results Display:** Present data in a dual-column format for direct comparison, enhancing the analytical experience by juxtaposing related insights.

In [13]:
def european_format(val):
    """
    Convert a numerical value to a formatted string as a currency representation in Euros.

    Parameters:
    val (float|int): Numeric value to format.

    Returns:
    str: Formatted string with the currency symbol and proper thousand and decimal separators.
    """
    if isinstance(val, float):
        # Format the float to two decimal places, using comma for decimal and period for thousands
        return f"€{val:,.2f}".replace(",", "@").replace(".", ",").replace("@", ".")
    else:
        # Return the integer or non-float value formatted to two decimal places without currency
        return f"{val:.2f}"

In [14]:
# Define mortgage details for two different scenarios
mortgages = [
    {"principal": principal_mortgage1, "rate": annual_rate_percent_mortgage1, "term_years": term_years_mortgage1, "extra_payment": extra_monthly_payment},
    {"principal": principal_mortgage2, "rate": annual_rate_percent_mortgage2, "term_years": term_years_mortgage2, "extra_payment": extra_monthly_payment}
]

# Initialize lists to hold calculation results and details
summary_data = []
schedules = []
labels = []
titles = []
result_messages = []
initial_details_messages = []

# Process each mortgage defined in the list
for index, mortgage in enumerate(mortgages):
    # Calculate the monthly payment based on principal, term, and interest rate    
    monthly_payment = calc_monthly_payment(mortgage["principal"], mortgage["term_years"], mortgage["rate"])

    # Generate the amortization schedule with possible extra payments and lump sum    
    schedule, schedule_offset = amortization_schedule(
        mortgage["principal"], mortgage["term_years"], mortgage["rate"], monthly_payment, mortgage["extra_payment"], lump_sum
    )

    # Calculate the total interest saved with and without extra payments    
    interest_saved_without, new_duration_without = calculate_total_interest_saved(
        mortgage["principal"], mortgage["rate"], mortgage["term_years"], monthly_payment
    )
    interest_saved_with, new_duration_with = calculate_total_interest_saved(
        mortgage["principal"], mortgage["rate"], mortgage["term_years"], monthly_payment, mortgage["extra_payment"], lump_sum
    )

    # Calculate the remaining mortgage balance after considering all payments and savings    
    final_balance = amortization_schedule(
            mortgage["principal"], mortgage["term_years"], mortgage["rate"], monthly_payment, mortgage["extra_payment"], lump_sum
        )[-1][-1]  
    remaining_mortgage = calculate_remaining_mortgage(mortgage["principal"], lump_sum)

    # Determine the reduction in years due to extra payments
    reduction_years = mortgage["term_years"] - new_duration_with

    # Collect amortization schedules for visualization
    schedules.append((schedule, schedule_offset))
    labels.append(('Current Mortgage', 'With Overpayment'))
    titles.append(f'Mortgage {index + 1} Amortization')

    # Prepare data for summary display in a DataFrame    
    summary_data.append({
        'Monthly Payment': monthly_payment,
        'Remaining Mortgage': remaining_mortgage,
        'Reduction Years': reduction_years,
        'Total Interest Saved': interest_saved_with,
        'Remaining Mortgage Years': new_duration_with
    })

    # Collect initial mortgage details for informative display    
    initial_details_messages.append(f"Mortgage {index + 1} Initial Details:\n"
                                    f"Lump Sum: {european_format(lump_sum)}\n"
                                    f"Extra Monthly Payment: {european_format(mortgage['extra_payment'])}\n"
                                    f"Original Balance: {european_format(mortgage['principal'])}\n"
                                    f"Current Interest Rate: {mortgage['rate']:.2f}%\n"
                                    f"Term: {mortgage['term_years']} years\n")

    # Summarize the results of the mortgage calculations for easy review
    result_messages.append(f"Mortgage {index + 1} Results:\n"
                           f"Monthly Payment: €{monthly_payment:.2f}\n"
                           f"Duration without Extras: {new_duration_without:.2f} years, Interest Saved: €{interest_saved_without:.2f}\n"
                           f"Duration with Extras: {new_duration_with:.2f} years, Interest Saved: €{interest_saved_with:.2f}\n")


In [15]:
# Creating a DataFrame to display the summarized data, structured for easy comparison between mortgages.
results_df = pd.DataFrame(summary_data, index=["Mortgage 1", "Mortgage 2"])
results_df.columns = [
    'Monthly Payment',
    'Remaining Mortgage',
    'Reduction Years',
    'Total Interest Saved',
    'Remaining Mortgage Years'
]

In [16]:
# Applying custom formatting to the DataFrame for better readability and presentation. This enhances the visual appeal and clarity of the data.
styled_results = results_df.style.format({
    'Monthly Payment': european_format,
    'Remaining Mortgage': european_format,
    'Total Interest Saved': european_format,
    'Remaining Mortgage Years': '{:.2f}',
    'Reduction Years': '{:.2f}'
}).set_properties(**{'text-align': 'right'})

### Expected Outcomes:

Users will gain a thorough understanding of their mortgage options, supported by detailed numerical summaries and interactive visual tools. This clarity empowers informed decision-making tailored to financial goals.

In [17]:
def format_initial_details_as_html(lump_sum, extra_payment, mortgages):
    """
    Generate HTML content for displaying initial mortgage details with common financial details.
    
    Parameters:
    lump_sum (float): Lump sum payment applied to the mortgage.
    extra_payment (float): Extra monthly payment applied to the mortgage.
    mortgages (list of dict): List of mortgage dictionaries containing principal, rate, and term_years.

    Returns:
    str: Formatted HTML string for mortgage initial details.
    """
    # Common details formatted as HTML list items, directly under the column title
    common_details = (f"<div>Lump Sum: {european_format(lump_sum)}<br>"
                      f"Extra Monthly Payment: {european_format(extra_payment)}</div>")
    # Start the HTML for mortgage details with a header
    mortgage_info_html = "<ul>"
    # Iterate through each mortgage to format its details into HTML list items
    for index, details in enumerate(mortgages, start=1):
        mortgage_info_html += (f"<li>Mortgage {index} Initial Details:"
                               f"<ul>"
                               f"<li>Original Balance: {european_format(details['principal'])}</li>"
                               f"<li>Current Interest Rate: {details['rate']:.2f}%</li>"
                               f"<li>Term: {details['term_years']} years</li>"
                               f"</ul>"
                               f"</li>")
    # Close the HTML list tag
    mortgage_info_html += "</ul>"
    return f"<div><h2>Initial Mortgage Details</h2>{common_details}{mortgage_info_html}</div>"

def format_results_as_html(result_messages):
    """
    Format mortgage result messages into an HTML string for display.

    Parameters:
    result_messages (list of str): List of formatted result messages for each mortgage.

    Returns:
    str: Formatted HTML string for displaying mortgage results.
    """
    # Start the HTML for mortgage results with a header
    results_html = "<h2>Mortgage Results</h2><ul>"
    # Convert each result message into HTML list item with proper line breaks
    for message in result_messages:
        formatted_message = message.replace('\n', '<br>')  # Replace newline with HTML line break
        results_html += f"<li>{formatted_message}</li>"
    # Close the HTML list tag
    results_html += "</ul>"
    return results_html

def display_two_columns(left_content, right_content):
    """
    Display two columns of HTML content side by side.

    Parameters:
    left_content (str): HTML content to display on the left column.
    right_content (str): HTML content to display on the right column.
    """
    # Define the HTML structure for a two-column layout
    html = f"""
    <div style='display: flex; width: 100%;'>
        <div style='width: 50%; padding: 10px;'>{left_content}</div>
        <div style='width: 50%; padding: 10px;'>{right_content}</div>
    </div>
    """
    display(HTML(html))

# Results and Summary

After the calculations are performed, a summary of your mortgage details and potential savings will be displayed. You will also see a comparison table that highlights the differences between the two mortgages with respect to the total interest paid and the reduction in the mortgage term.

In [18]:
# Generate HTML for initial details and results
initial_details_html = format_initial_details_as_html(lump_sum, extra_monthly_payment, mortgages)
results_html = format_results_as_html(result_messages)
display_two_columns(initial_details_html, results_html)

# Display the DataFrame with applied style if needed
display(styled_results)

# Plotting the visualization
plot_amortization_interactive(schedules, labels, titles)

,Monthly Payment,Remaining Mortgage,Reduction Years,Total Interest Saved,Remaining Mortgage Years
Mortgage 1,"€958,77","€74.104,00",5.83,"€17.281,83",7.17
Mortgage 2,"€1.866,80","€330.000,00",5.50,"€73.865,96",21.50


# Return on Investment (ROI) Analysis
In this section, we analyze the return on investment for extra payments made towards the mortgage. We calculate various forms of ROI to assess the efficiency and effectiveness of the investment in terms of interest savings over the lifetime of the mortgage.

## Types of ROI Calculated
- ### Simple ROI
  - **Definition:** Simple ROI measures the efficiency of an investment without adjusting for time. It provides a quick snapshot of the overall effectiveness of the additional payments compared to the initial investment.
  - **Calculation:**
Simple ROI = ((Total Interest Saved - Total Extra Payments) / Total Extra Payments) * 100

- ### Annualized ROI
  - **Definition:** This calculation annualizes the return, allowing you to compare the efficiency of your mortgage investments against other investments that also report annual returns. It is useful for evaluating investment performance adjusted for time.
  - **Calculation:**
Annualized ROI = ((Total Interest Saved - Total Extra Payments) / Total Extra Payments / Number of Years) * 100

- ### CAGR (Compound Annual Growth Rate)
  - **Definition:** Provides a compound annual growth rate, which is particularly useful for understanding the performance of an investment over time considering the effect of compound interest. This is crucial if we consider the interest savings as a continuous reinvestment.
  - **Calculation:**
CAGR = (((Final Value / Initial Investment) ** (1 / Number of Years)) - 1) * 100. Where *Final Value* includes the total interest saved plus the initial lump sum, and *Initial Investment* is the lump sum payment.

- ### Linear Annual ROI
  - **Definition:** Similar to Annualized ROI but focuses on distributing the return linearly over the investment years. This approach assumes a constant return each year, which simplifies the analysis but may not capture the actual variability year-over-year.
  - **Calculation:**
Linear Annual ROI = ((Total Interest Saved - Total Extra Payments) / (Total Extra Payments * Number of Years)) * 100

## Application
These ROI metrics will help us understand not only the immediate benefits but also the long-term advantages of making extra payments on the mortgage. By quantifying the savings and comparing them to other potential investments, homeowners can make informed decisions about where to allocate their funds for maximum financial benefit.

In [19]:
def calculate_simple_roi(interest_saved, total_extra_payments):
    """
    Calculate the Simple Return on Investment (ROI).

    Parameters:
    interest_saved (float): The total interest amount saved due to extra payments.
    total_extra_payments (float): The total amount of extra payments made.

    Returns:
    float: Simple ROI percentage. Returns 'inf' if no extra payments were made but savings occurred.
    """
    if total_extra_payments == 0:
        return float('inf')  # Infinite return if no extra payments were made.
    return ((interest_saved - total_extra_payments) / total_extra_payments) * 100

def calculate_annualized_roi(interest_saved, total_extra_payments, years):
    """
    Calculate the Annualized Return on Investment (ROI), which provides a normalized measure of profitability over a year.

    Parameters:
    interest_saved (float): The total interest amount saved.
    total_extra_payments (float): The total amount of extra payments made.
    years (float): The number of years over which the ROI is calculated.

    Returns:
    float: Annualized ROI percentage. Returns 'inf' if no extra payments were made.
    """
    if total_extra_payments == 0:
        return float('inf')
    return ((interest_saved - total_extra_payments) / total_extra_payments / years) * 100

def calculate_cagr(final_value, initial_value, years):
    """
    Calculate the Compound Annual Growth Rate (CAGR), a useful measure of growth over multiple time periods.

    Parameters:
    final_value (float): The end value of the investment after the specified period.
    initial_value (float): The start value of the investment.
    years (float): The number of years over which the growth is calculated.

    Returns:
    float: CAGR percentage. Returns 'inf' if the initial investment was zero.
    """
    if initial_value == 0:
        return float('inf')
    return ((final_value / initial_value) ** (1 / years) - 1) * 100

def calculate_linear_annual_roi(total_interest_saved, total_extra_payments, years):
    """
    Calculate the Linear Annual Return on Investment (ROI), distributing the return evenly over the years.

    Parameters:
    total_interest_saved (float): The total interest saved over the period.
    total_extra_payments (float): The total extra payments made.
    years (float): The number of years over which the ROI is calculated.

    Returns:
    float: Linear Annual ROI percentage. Returns 'inf' if no extra payments were made.
    """
    if total_extra_payments == 0:
        return float('inf')
    return ((total_interest_saved - total_extra_payments) / (total_extra_payments * years)) * 100

In [20]:
# Calculate monthly payments for both mortgages using pre-defined loan details
monthly_payment_mortgage1 = calc_monthly_payment(principal_mortgage1, term_years_mortgage1, annual_rate_percent_mortgage1)
monthly_payment_mortgage2 = calc_monthly_payment(principal_mortgage2, term_years_mortgage2, annual_rate_percent_mortgage2)

# Calculate the total interest saved and new loan duration considering extra payments for Mortgage 1
interest_saved_1, new_duration_with_1 = calculate_total_interest_saved(
    principal_mortgage1, annual_rate_percent_mortgage1, term_years_mortgage1, monthly_payment_mortgage1, extra_monthly_payment, lump_sum)

# Compute total extra payments made over the new duration of the loan for Mortgage 1
total_extra_payments_1 = extra_monthly_payment * new_duration_with_1 * 12 + lump_sum

# Calculate various ROI metrics for Mortgage 1 using the interest saved and total extra payments
simple_roi_1 = calculate_simple_roi(interest_saved_1, total_extra_payments_1)
annualized_roi_1 = calculate_annualized_roi(interest_saved_1, total_extra_payments_1, new_duration_with_1)
cagr_1 = calculate_cagr(interest_saved_1 + lump_sum, lump_sum, new_duration_with_1)
linear_annual_roi_1 = calculate_linear_annual_roi(interest_saved_1, total_extra_payments_1, new_duration_with_1)

# Repeat the calculations for Mortgage 2
interest_saved_2, new_duration_with_2 = calculate_total_interest_saved(
    principal_mortgage2, annual_rate_percent_mortgage2, term_years_mortgage2, monthly_payment_mortgage2, extra_monthly_payment, lump_sum)
total_extra_payments_2 = extra_monthly_payment * new_duration_with_2 * 12 + lump_sum

simple_roi_2 = calculate_simple_roi(interest_saved_2, total_extra_payments_2)
annualized_roi_2 = calculate_annualized_roi(interest_saved_2, total_extra_payments_2, new_duration_with_2)
cagr_2 = calculate_cagr(interest_saved_2 + lump_sum, lump_sum, new_duration_with_2)
linear_annual_roi_2 = calculate_linear_annual_roi(interest_saved_2, total_extra_payments_2, new_duration_with_2)

In [21]:
def analyze_roi_results(interest_saved, total_extra_payments, years):
    """
    Analyze and summarize different ROI metrics for a mortgage investment.

    Parameters:
    interest_saved (float): The total amount of interest saved by making extra payments.
    total_extra_payments (float): The total extra payments made over the life of the mortgage.
    years (float): The duration over which the extra payments were made.

    Returns:
    str: A string summarizing the ROI from various calculations.
    """
    analysis = ""
    simple_roi = calculate_simple_roi(interest_saved, total_extra_payments)
    annualized_roi = calculate_annualized_roi(interest_saved, total_extra_payments, years)
    cagr = calculate_cagr(interest_saved + total_extra_payments, total_extra_payments, years)
    linear_annual_roi = calculate_linear_annual_roi(interest_saved, total_extra_payments, years)

    # Analysis of Simple ROI
    if simple_roi < 0:
        analysis += f"Simple ROI: {simple_roi:.2f}% - Indicates a loss, suggesting that the additional payments do not offset the initial costs plus the interest saved.\n"
    else:
        analysis += f"Simple ROI: {simple_roi:.2f}% - Shows a positive return, indicating a net benefit relative to the initial investment.\n"

    # Analysis of Annualized ROI
    if annualized_roi < 0:
        analysis += f"Annualized ROI: {annualized_roi:.2f}% - Suggests that, when adjusted annually, the investment does not yield profits, reflecting possible inefficiency over the evaluated term.\n"
    else:
        analysis += f"Annualized ROI: {annualized_roi:.2f}% - Positive, reflects an effective annual profitability, favorably comparing it with other investments.\n"

    # Analysis of CAGR
    analysis += f"CAGR: {cagr:.2f}% - Represents the annual compound growth. A positive CAGR indicates an effective increase in the value of the investment over time, considering the effect of compound interest.\n"

    # Analysis of Linear Annual ROI
    if linear_annual_roi < 0:
        analysis += f"Linear Annual ROI: {linear_annual_roi:.2f}% - Negative, indicates uniform losses each year, which could discourage this payment strategy if a consistent return is expected.\n"
    else:
        analysis += f"Linear Annual ROI: {linear_annual_roi:.2f}% - Positive, shows gains evenly distributed over the years, ideal for those seeking stability in annual returns.\n"

    return analysis

# Example usage of the function with actual calculated data for each mortgage
analysis_mortgage1 = analyze_roi_results(interest_saved_1, total_extra_payments_1, new_duration_with_1)
analysis_mortgage2 = analyze_roi_results(interest_saved_2, total_extra_payments_2, new_duration_with_2)

print("ROI Analysis for Mortgage 1:")
print(analysis_mortgage1)
print("\nROI Analysis for Mortgage 2:")
print(analysis_mortgage2)

ROI Analysis for Mortgage 1:
Simple ROI: -65.44% - Indicates a loss, suggesting that the additional payments do not offset the initial costs plus the interest saved.
Annualized ROI: -9.13% - Suggests that, when adjusted annually, the investment does not yield profits, reflecting possible inefficiency over the evaluated term.
CAGR: 4.23% - Represents the annual compound growth. A positive CAGR indicates an effective increase in the value of the investment over time, considering the effect of compound interest.
Linear Annual ROI: -9.13% - Negative, indicates uniform losses each year, which could discourage this payment strategy if a consistent return is expected.


ROI Analysis for Mortgage 2:
Simple ROI: 47.73% - Shows a positive return, indicating a net benefit relative to the initial investment.
Annualized ROI: 2.22% - Positive, reflects an effective annual profitability, favorably comparing it with other investments.
CAGR: 4.31% - Represents the annual compound growth. A positive CAG

# Amortization Schedule Analysis
In this section, we present the amortization schedules for the mortgages under consideration. The schedules take into account the principal amounts, interest rates, terms, and any extra payments or lump sums. These schedules help in understanding how each payment affects the loan balance over time, and they provide a clear view of the amortization process under the French amortization system.

## Overview of Amortization Schedule
The amortization schedule details each payment's impact on the loan balance, showing the split between principal and interest, as well as the remaining balance after each payment. By incorporating extra payments and lump sum contributions, we can observe the accelerated reduction in principal, which results in significant savings on interest over the loan's lifetime.

### Key Components of the Amortization Schedule:
- **Total Payment**: The total amount paid monthly, including both principal and interest.
- **Interest**: The interest component of the monthly payment.
- **Principal**: The portion of the monthly payment that goes towards reducing the loan balance.
- **Remaining Balance**: The outstanding balance of the loan after each payment.
- **Remaining Years**: The number of years left until the loan is fully paid off, recalculated each month.

### Method Used:
- **French Amortization Method**: This method ensures that the total payment amount is fixed (assuming no changes to the interest rate or loan terms), with the interest component decreasing and the principal component increasing over time.

## Amortization Tables
Below are the generated amortization tables for Mortgage 1 and Mortgage 2, including details about the monthly payments, interest and principal parts, and the decreasing loan balances over the years.

### Mortgage 1 Amortization Table
This table starts from the current year and details each payment's effect under the terms agreed upon for Mortgage 1, including any extra payments and the impact of the initial lump sum.

In [22]:
def generate_amortization_schedule(principal, interest_rate, term_years, monthly_payment, extra_monthly_payment=0, lump_sum=0):
    """
    Generates an amortization schedule based on the French amortization method, accounting for potential extra payments and a lump sum payment. The schedule includes detailed payment information for each month over the term of the loan, with calculations reflecting the impact of both the extra payments and the lump sum.

    Parameters:
    - principal (float): The initial loan balance.
    - interest_rate (float): The annual interest rate as a percentage.
    - term_years (int): The total duration of the loan in years.
    - monthly_payment (float): The standard monthly payment amount calculated without any extra payments.
    - extra_monthly_payment (float, optional): Any additional amount added to the standard monthly payment to reduce the principal faster. Defaults to 0.
    - lump_sum (float, optional): An upfront payment made at the beginning of the loan period to immediately reduce the principal. Defaults to 0.

    Returns:
    - pd.DataFrame: A DataFrame with an index set to the 'Year' of each payment and columns for 'Month', 'Total Payment', 'Interest', 'Principal', 'Remaining Balance', and 'Remaining Years'. This structure provides a comprehensive view of how each payment affects the loan balance and how quickly the loan is being paid off over time.

    The function calculates the monthly interest rate from the annual rate and iterates through each payment period to compute the interest, principal part of each payment, and the remaining balance. It adjusts for any extra monthly payments or lump sum payments, recalculating the remaining balance and future payments accordingly. If the remaining balance of the loan reaches zero before the end of the term, the loop exits early.
    """
    # Get the current year to base the schedule's years off of
    current_year = datetime.datetime.now().year
    
    # Convert the annual interest rate from a percentage to a decimal
    interest_rate /= 100
    
    # Determine the monthly interest rate from the annual interest rate
    monthly_interest_rate = interest_rate / 12
    
    # Initialize the list to hold each payment's details
    payments = []
    
    # Adjust the initial loan balance by subtracting any lump sum payment
    current_balance = principal - lump_sum
    
    # Calculate the total number of payments to be made
    total_months = term_years * 12

    # Loop through each month up to the total number of payments
    for month in range(1, total_months + 1):
        # If the current balance is less than or equal to zero, break out of the loop early
        if current_balance <= 0:
            break
        
        # Calculate the amount of interest for this month's payment
        interest_payment = current_balance * monthly_interest_rate
        
        # Total payment for the month is the standard payment plus any extra payment
        total_payment = monthly_payment + extra_monthly_payment
        
        # The principal payment is the total payment minus the interest component
        principal_payment = total_payment - interest_payment
        
        # Subtract the principal payment from the current balance to get the new balance
        current_balance -= principal_payment

        # Calculate the year for this month's payment based on the current year and the month number
        payment_year = current_year + (month - 1) // 12

        # Adjust the principal and interest if the current balance falls below zero
        if current_balance < 0:
            principal_payment += current_balance
            interest_payment -= current_balance
            current_balance = 0

        # Add the details of this month's payment to the list
        payments.append({
            "Year": payment_year,
            "Month": month,
            "Total Payment": '{:,.2f}'.format(total_payment),
            "Interest": '{:,.2f}'.format(interest_payment),
            "Principal": '{:,.2f}'.format(principal_payment),
            "Remaining Balance": '{:,.2f}'.format(current_balance),
            "Remaining Years": '{:,.2f}'.format((total_months - month + 1) / 12)
        })

    # Convert the list of payments into a DataFrame for easier viewing and manipulation
    df = pd.DataFrame(payments)
    df.set_index('Year', inplace=True)  # Set 'Year' as the DataFrame index to organize the data by year

    return df

# Calculating the monthly payments using the provided function or values
monthly_payment_m1 = calc_monthly_payment(principal_mortgage1, term_years_mortgage1, annual_rate_percent_mortgage1)
monthly_payment_m2 = calc_monthly_payment(principal_mortgage2, term_years_mortgage2, annual_rate_percent_mortgage2)

# Generating amortization schedules for both mortgages
amortization_table_m1 = generate_amortization_schedule(
    principal_mortgage1, annual_rate_percent_mortgage1, term_years_mortgage1, monthly_payment_m1, extra_monthly_payment, lump_sum)

amortization_table_m2 = generate_amortization_schedule(
    principal_mortgage2, annual_rate_percent_mortgage2, term_years_mortgage2, monthly_payment_m2, extra_monthly_payment, lump_sum)

# Displaying the first few rows of the amortization tables
display(amortization_table_m1)
display(amortization_table_m2)

,Month,Total Payment,Interest,Principal,Remaining Balance,Remaining Years
Year,,,,,,
2024,1,958.77,182.17,776.60,"73,327.40",13.00
2024,2,958.77,180.26,778.51,"72,548.89",12.92
2024,3,958.77,178.35,780.43,"71,768.46",12.83
2024,4,958.77,176.43,782.34,"70,986.12",12.75
2024,5,958.77,174.51,784.27,"70,201.85",12.67
...,...,...,...,...,...,...
2030,82,958.77,11.29,947.48,"3,645.69",6.25
2030,83,958.77,8.96,949.81,"2,695.88",6.17
2030,84,958.77,6.63,952.15,"1,743.73",6.08


,Month,Total Payment,Interest,Principal,Remaining Balance,Remaining Years
Year,,,,,,
2024,1,"1,866.80","1,031.25",835.55,"329,164.45",27.00
2024,2,"1,866.80","1,028.64",838.16,"328,326.30",26.92
2024,3,"1,866.80","1,026.02",840.78,"327,485.52",26.83
2024,4,"1,866.80","1,023.39",843.40,"326,642.11",26.75
2024,5,"1,866.80","1,020.76",846.04,"325,796.07",26.67
...,...,...,...,...,...,...
2045,254,"1,866.80",26.87,"1,839.92","6,759.99",5.92
2045,255,"1,866.80",21.12,"1,845.67","4,914.31",5.83
2045,256,"1,866.80",15.36,"1,851.44","3,062.88",5.75
